In [7]:
file= '/home/caow/03mnv/analyse3/02data/01origin/hg38mnv'
url='/home/caow/03mnv/analyse3/02data/02annotation/'
dataset='1000G,GTEx'
maf_switch=0 # 如果是1的话，启用MAF过滤
dataset=dataset.split(',')

In [8]:
mnv=[]
with open(file) as f:
    for i in f:
        a=i.strip().split('\t')
        b=a[-1]
        # if 'TCGA' in b or 'GTEx' in b or '1000G' in b or 'UKB20w' in b or 'UKB50w' in b:
        #   mnv.append(a[2])
        flag=0
        for ii in dataset:
            if ii in b:
                flag=1
                break
        
        if maf_switch==1:
            if flag==1:
                for ii in b.split('|'):
                    if ii.startswith(dataset[0]):
                        ac=int(ii.split('/')[0].split(':')[1])
                        break
                if ac>3: # 相当于maf=0.05%
                    mnv.append(a[2])
        else:
            if flag==1:
                mnv.append(a[2])

mnv=set(mnv)

In [3]:
def createGeneAnno():
    anno={}
    anno['UTR5'] = 0
    anno['exon'] = 0
    anno['splice'] = 0
    anno['intron'] = 0
    anno['UTR3'] = 0
    return anno
anno=createGeneAnno()

UTR5 = []
exon = []
splice = []
intron = []
UTR3 = []

with open(url+'gene.txt') as f:
    for i in f:
        a=i.strip().split('\t')
        if a[-1] !='.' and a[2] in mnv:
            if 'UTR5_variant' in a[-1]:
                UTR5.append(a[2])
            if 'exon' in a[-1]:
                exon.append(a[2])
            if 'splice_' in a[-1]:
                splice.append(a[2])
            if 'intron_variant' in a[-1]:
                intron.append(a[2])
            if 'UTR3_variant' in a[-1]:
                UTR3.append(a[2])   

In [4]:
UTR5 = set(UTR5)
exon = set(exon)
splice = set(splice)
intron = set(intron)
UTR3 = set(UTR3)

anno['UTR5']=len(UTR5)
anno['exon']=len(exon)
anno['splice']=len(splice)
anno['intron']=len(intron)
anno['UTR3']=len(UTR3)

# lncRNA
lnc=[]
f=open(url+'lnc.txt')
for i in f:
    a=i.strip().split('\t')
    if a[2] in mnv and 'all lncRNA' in a[-1]:
        lnc.append(a[2])
f.close()
lnc=set(lnc)
anno['lncRNA']=len(lnc)

# miR 
miR=[]
f=open(url+'pre_miRNA.txt')
for i in f:
    a=i.strip().split('\t')
    if a[2] in mnv and 'all miRNA' in a[-1]:
        miR.append(a[2])
f.close()
miR=set(miR)
anno['miRNA']=len(miR)

# circRNA
circRNA=[]
f=open(url+'circ.txt')
for i in f:
    a=i.strip().split('\t')
    if a[2] in mnv and 'all circRNA' in a[-1]:
        circRNA.append(a[2])
f.close()
circRNA=set(circRNA)
anno['circRNA']=len(circRNA)

# piRNA
piRNA=[]
f=open(url+'piR.txt')
for i in f:
    a=i.strip().split('\t')
    if a[2] in mnv and 'all piRNA' in a[-1]:
        piRNA.append(a[2])
f.close()
piRNA=set(piRNA)
anno['piRNA']=len(piRNA)

# ATAC
ATAC=[]
f=open(url+'atac.txt')
for i in f:
    a=i.strip().split('\t')
    if a[2] in mnv and 'all ATAC' in a[-1]:
        ATAC.append(a[2])
f.close()
ATAC=set(ATAC)
anno['ATAC']=len(ATAC)

# CE
conserved_element=[]
f=open(url+'CE.txt')
for i in f:
    a=i.strip().split('\t')
    if a[2] in mnv and 'all conserved_element' in a[-1]:
        conserved_element.append(a[2])
f.close()
conserved_element=set(conserved_element)
anno['conserved_element']=len(conserved_element)

# enhancer
enhancer=[]
f=open(url+'enhancer.txt')
for i in f:
    a=i.strip().split('\t')
    if a[2] in mnv and 'all enhancer' in a[-1]:
        enhancer.append(a[2])
f.close()
enhancer=set(enhancer)
anno['enhancer']=len(enhancer)

# miRBS
miRBS=[]
f=open(url+'miRBS.txt')
for i in f:
    a=i.strip().split('\t')
    if a[2] in mnv and 'all miRBS' in a[-1]:
        miRBS.append(a[2])
f.close()
miRBS=set(miRBS)
anno['miRBS']=len(miRBS)

# TFBS
TFBS=[]
f=open(url+'TFBS.txt')
for i in f:
    a=i.strip().split('\t')
    if a[2] in mnv and 'all TFBS' in a[-1]:
        TFBS.append(a[2])
f.close()
TFBS=set(TFBS)
anno['TFBS']=len(TFBS)

In [5]:
intergenic=mnv-UTR5-exon-splice-intron-UTR3-lnc-miR-circRNA-piRNA


In [9]:
from collections import defaultdict

# mnv: set of MNV ids (你前面已经算出来了)
# url: annotation dir

mnv2anno = defaultdict(set)

def add_anno_from_file(path, anno_func):
    """
    path: 注释文件路径
    anno_func: 输入该行 split 后的数组 a，返回一个或多个注释字符串（list/tuple/set），或返回空表示不记
    """
    with open(path) as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            a = line.split('\t')
            if len(a) < 3:
                continue
            mnvid = a[2]
            if mnvid not in mnv:
                continue

            annos = anno_func(a)
            if not annos:
                continue
            if isinstance(annos, str):
                mnv2anno[mnvid].add(annos)
            else:
                for x in annos:
                    if x:
                        mnv2anno[mnvid].add(x)

# gene 区域：可能一条记录同时命中多个（按你原脚本的 if 逻辑）
def gene_annos(a):
    cons = a[-1]
    if cons == '.':
        return []
    out = []
    if 'UTR5_variant' in cons:
        out.append('UTR5')
    if 'exon' in cons:
        out.append('exon')
    if 'splice_' in cons:
        out.append('splice')
    if 'intron_variant' in cons:
        out.append('intron')
    if 'UTR3_variant' in cons:
        out.append('UTR3')
    return out

add_anno_from_file(url + 'gene.txt', gene_annos)

# 其他大类：按原逻辑，只要包含某个“all xxx”标签就打这个注释
add_anno_from_file(url + 'lnc.txt',        lambda a: 'lncRNA' if 'all lncRNA' in a[-1] else [])
add_anno_from_file(url + 'mature_miRNA.txt',  lambda a: 'mature_miRNA'  if 'all miRNA' in a[-1] else [])
add_anno_from_file(url + 'circ.txt',       lambda a: 'circRNA' if 'all circRNA' in a[-1] else [])
add_anno_from_file(url + 'piR.txt',        lambda a: 'piRNA' if 'all piRNA' in a[-1] else [])
add_anno_from_file(url + 'atac.txt',       lambda a: 'ATAC' if 'all ATAC' in a[-1] else [])
add_anno_from_file(url + 'CE.txt',         lambda a: 'conserved_element' if 'all conserved_element' in a[-1] else [])
add_anno_from_file(url + 'enhancer.txt',   lambda a: 'enhancer' if 'all enhancer' in a[-1] else [])
add_anno_from_file(url + 'miRBS.txt',      lambda a: 'miRBS' if 'all miRBS' in a[-1] else [])
add_anno_from_file(url + 'TFBS.txt',       lambda a: 'TFBS' if 'all TFBS' in a[-1] else [])

# intergenic：不在任何注释集合中的 mnv
for mnvid in mnv:
    if mnvid not in mnv2anno:
        mnv2anno[mnvid].add('intergenic')

# 输出 TSV：两列 MNVid / annotation（多注释用 ; 连接）
out_tsv = '/home/caow/03mnv/analyse3/00addition/data/1000G_GTEx_annotation.tsv'
with open(out_tsv, 'w') as out:
    out.write("MNVid\tannotation\n")
    for mnvid in sorted(mnv2anno.keys()):
        annostr = ";".join(sorted(mnv2anno[mnvid]))
        out.write(f"{mnvid}\t{annostr}\n")

In [7]:
2429195/(63993320+43066422)

0.02269008830602263